In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from pyod.models.copod import COPOD
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize


In [2]:
df = pd.read_csv('train.csv')

In [3]:
df_test = pd.read_csv('test.csv')[["id", "all"]]
len(df_test)
x_df_test = df_test['all']

In [4]:
x_df = df['all']
x_df_temp = x_df._append(x_df_test)
y_df = df['label']
x_df_temp

0       MAJOR LIBERAL RAG RELUCTANTLY PUBLISHES Articl...
1       AT&T, Time Warner and the Death of PrivacyAT&T...
2       The World Sees a Diminished AmericaThe World S...
3       The Bolton Threat to Trump’s Middle East Polic...
4       Life at migrant centers in Germany: RT talks t...
                              ...                        
7883    Italy's ruling PD slides further in polls as e...
7884    Trump-backed Navy expansion would boost costs ...
7885    Canada and E.U. Sign Trade Deal, Bucking Resis...
7886    With Koch Brothers Academy, Conservatives Sett...
7887    More than 50 arrested for looting in Miami dur...
Name: all, Length: 17438, dtype: object

In [5]:
count_vectorizer = CountVectorizer(max_features=60000)
count_vectorizer.fit_transform(x_df_temp)
freq_term_matrix = count_vectorizer.transform(x_df_temp)
tfidf = TfidfTransformer(norm = "l2")
tfidf.fit(freq_term_matrix)
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)


In [6]:
tf_idf_matrix

<17438x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 4457207 stored elements in Compressed Sparse Row format>

In [7]:
x_train, x_test, y_train, y_test = train_test_split(tf_idf_matrix[:9550],y_df,random_state=0)

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tf_idf_matrix[:9550],y_df)
#Predict on the test set and calculate accuracy
y_pred=pac.predict(x_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 100.0%


In [15]:
freq_term_matrix_test = count_vectorizer.transform(x_df_test)
tf_idf_matrix_test = tfidf.transform(freq_term_matrix_test)

In [16]:
tf_idf_matrix_test.shape

(7888, 138439)

In [17]:
df_out = pd.DataFrame()
df_out['id'] = df_test['id']

In [18]:
y_pred_pac_test = pac.predict(tf_idf_matrix_test)
df_out["label"] = y_pred_pac_test

In [19]:
df_out.to_csv('submission.csv',index=False)

In [ ]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(x_train,y_train)
#Predict on the test set and calculate accuracy
y_pred=xgb.predict(x_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

In [ ]:
df_out = pd.DataFrame()
df_out['id'] = df_test['id']
y_pred_xgb_test = xgb.predict(tf_idf_matrix_test)
df_out["label"] = y_pred_xgb_test
df_out.to_csv('submission.csv', index=False)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB(alpha=.01,)
clf.fit(x_train,y_train) 
y_pred_clf = clf.predict(x_test)
score=accuracy_score(y_test,y_pred_clf)
print(f'Accuracy: {round(score*100,2)}%')

In [8]:
clf_name = 'COPOD'
clf = COPOD()

# you could try parallel version as well.
clf = COPOD()
clf.fit(x_train.toarray(),y_train)

C:\Akshay\Tools\Python\Lib\site-packages\pyod\models\base.py:430: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(


COPOD(contamination=0.1, n_jobs=1)

In [11]:
y_pred_clf = clf.predict(x_test)
score=accuracy_score(y_test,y_pred_clf)
print(f'Accuracy: {round(score*100,2)}%')

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 0 dimension(s)

In [10]:
x_test

<2388x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 617717 stored elements in Compressed Sparse Row format>